In [1]:
%cd /kaggle/working

/kaggle/working


In [2]:
import os
os.environ['WANDB_MODE'] = 'disabled'

In [3]:
!git clone https://github.com/WongKinYiu/yolov9.git

Cloning into 'yolov9'...
remote: Enumerating objects: 704, done.
remote: Counting objects: 100% (330/330), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 704 (delta 246), reused 202 (delta 201), pack-reused 374
Receiving objects: 100% (704/704), 3.23 MiB | 17.06 MiB/s, done.
Resolving deltas: 100% (291/291), done.


In [4]:
%cd /kaggle/working/yolov9

/kaggle/working/yolov9


In [5]:
# !wget -P /kaggle/working/yolov9 https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt

# !wget -P /kaggle/working/yolov9 https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt

In [6]:
!conda install -y gdown

Retrieving notices: ...working... done
- \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

In [7]:
%cd /kaggle/working/
# dataset link for bangla sign language zip
# !gdown --id 1JQhewCk77RXOLryMsmDdc11q4YubHAhF
# dataset for testing
!gdown 1-4MU6v0g_I-Lm8l4wrfeNFO9xEoVNYwO

/kaggle/working
Downloading...
From (original): https://drive.google.com/uc?id=1-4MU6v0g_I-Lm8l4wrfeNFO9xEoVNYwO
From (redirected): https://drive.google.com/uc?id=1-4MU6v0g_I-Lm8l4wrfeNFO9xEoVNYwO&confirm=t&uuid=fe7a2280-849a-498f-80c5-3cb4cad98823
To: /kaggle/working/bdsl_test.zip
100%|████████████████████████████████████████| 880M/880M [00:13<00:00, 64.5MB/s]


In [8]:
import zipfile
import os



# Step 2: Define the path to the zip file
zip_file_path = '/kaggle/working/bdsl_test.zip'

# Step 3: Extract the name of the zip file (without extension) to create a folder
zip_file_name = os.path.basename(zip_file_path)
folder_name = os.path.splitext(zip_file_name)[0]
extract_to_path = os.path.join('/kaggle/working/yolov9', folder_name)

# Step 4: Create the output directory if it doesn't exist
os.makedirs(extract_to_path, exist_ok=True)

# Step 5: Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

# Verify the extraction
if os.path.exists(extract_to_path):
    print(f"Folder successfully unzipped to {extract_to_path}")
else:
    print("Failed to unzip the folder.")

Folder successfully unzipped to /kaggle/working/yolov9/bdsl_test


In [9]:
import os

file_path = '/kaggle/working/bdsl_test.zip'

# Check if the file exists before trying to delete it
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"{file_path} has been deleted")
else:
    print(f"{file_path} does not exist")


/kaggle/working/bdsl_test.zip has been deleted


In [10]:
# %cd /kaggle/working/yolov9/models/detect
# !gdown --id 1SHNpPjhxI11Cu3Lz8ZN5lhCDLe4okBCe

In [11]:
%cd /kaggle/working/yolov9
# data.yml file
# !gdown 1yRa2jbtUw74fhIr9o73hECeSPy86z52t
# yml file for shoreo shoreya
!gdown 14tHwlpaRImLNcEuo8JqW5SFnuXcTsCA1

/kaggle/working/yolov9
Downloading...
From: https://drive.google.com/uc?id=14tHwlpaRImLNcEuo8JqW5SFnuXcTsCA1
To: /kaggle/working/yolov9/data.yaml
100%|██████████████████████████████████████████| 222/222 [00:00<00:00, 1.01MB/s]


In [12]:
%cd /kaggle/working/yolov9

/kaggle/working/yolov9


In [13]:
#!python train_dual.py --workers 16 --batch 8 --img 640 --epochs 20 --data /kaggle/working/yolov9/data.yaml --weights /kaggle/working/yolov9/yolov9-e.pt --device 0,1 --cfg /kaggle/working/yolov9/models/detect/yolov9_custom.yaml --hyp /kaggle/working/yolov9/data/hyps/hyp.scratch-high.yaml

In [14]:
# !python train_dual.py \
# --workers 16 --batch 8 --img 512 --epochs 10 --verbose True \
# --resume true --data /kaggle/working/yolov9/data.yaml \
# --weights /kaggle/working/yolov9/yolov9-e.pt --device 0,1 \
# --cfg /kaggle/working/yolov9/models/detect/yolov9_custom.yaml \
# --hyp /kaggle/working/yolov9/data/hyps/hyp.scratch-high.yaml

### 

In [15]:
!pip install ultralytics 


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 745.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.6/779.6 kB 6.3 MB/s eta 0:00:00


In [16]:
%cd /kaggle/working/yolov9

/kaggle/working/yolov9


In [17]:
#best pt file 
# !gdown 1Pgf9D29jDo2RrY3EbjUWsBco0lbE0k4F

In [18]:
from ultralytics import YOLO


# Build a YOLOv9c model from pretrained weight
model = YOLO("yolov9c.pt")

# Display model information (optional)
model.info()



100%|██████████| 49.4M/49.4M [00:00<00:00, 223MB/s]


YOLOv9c summary: 618 layers, 25590912 parameters, 0 gradients, 104.0 GFLOPs


(618, 25590912, 0, 104.02268160000003)

In [19]:
import torch
import torch.nn as nn


class DataParallelModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.block1 = nn.Linear(10, 20)

        # wrap block2 in DataParallel
        self.block2 = nn.Linear(20, 20)
        self.block2 = nn.DataParallel(self.block2)

        self.block3 = nn.Linear(20, 20)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        return x

In [20]:
# train.py
import wandb
import random  # for demo script

wandb.login('957d38e93afcd97d0286c494f525efdb6caf7707')

epochs = 10
lr = 0.01

run = wandb.init(
    # Set the project where this run will be logged
    project="my-awesome-project",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": lr,
        "epochs": epochs,
    },
)


In [21]:
# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data="/kaggle/working/yolov9/data.yaml",  epochs=10, imgsz=640, verbose=True,save=True,save_period=1,device=[0,1])

Ultralytics YOLOv8.2.27 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov9c.pt, data=/kaggle/working/yolov9/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=[0, 1], workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_t

100%|██████████| 22.2M/22.2M [00:00<00:00, 137MB/s] 
2024-06-01 19:53:43,299	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-01 19:53:44,869	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    212864  ultralytics.nn.modules.block.RepNCSPELAN4    [128, 256, 128, 64, 1]        
  3                  -1  1    164352  ultralytics.nn.modules.block.ADown           [256, 256]                    
  4                  -1  1    847616  ultralytics.nn.modules.block.RepNCSPELAN4    [256, 512, 256, 128, 1]       
  5                  -1  1    656384  ultralytics.nn.modules.block.ADown           [512, 512]                    
  6                  -1  1   2857472  ultralytics.nn.modules.block.RepNCSPELAN4    [512, 512, 512, 256, 1]       
  7                  -1  1    656384  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 73.5MB/s]


AMP: checks passed ✅


...

                   all        390        390      0.896      0.951      0.969      0.844

10 epochs completed in 0.341 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 51.6MB
Optimizer stripped from runs/detect/train/weights/best.pt, 51.6MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.27 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
YOLOv9c summary (fused): 384 layers, 25324645 parameters, 0 gradients, 102.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:08<00:00,  2.95it/s]
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:465: UserWarning: Glyph 2437 (\N{BENGALI LETTER A}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:465: UserWarning: Matplotlib currently does not support Bengali natively.
  fig.savefig(save_dir, dpi=250)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:465: UserWarning: Glyph 2438 (\N{BENGALI LETTER AA}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:465: UserWarning: Glyph 2439 (\N{BENGALI LETTER I}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:465: UserWarning: Glyph 2442 (\N{BENGALI LETTER UU}) missing from current

                   all        390        390      0.896       0.95      0.969      0.844
                     অ         31         31      0.969      0.999       0.98      0.788
                     আ         59         59       0.86      0.935      0.962      0.778
                     ই         60         60      0.983      0.945      0.983      0.909
                     ঊ         60         60      0.665      0.958      0.939      0.826
                     এ         60         60      0.927      0.867      0.942      0.876
                     ও         60         60      0.869      0.983      0.981      0.843
                     ক         60         60          1      0.965      0.995       0.89


/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:427: UserWarning: Glyph 2437 (\N{BENGALI LETTER A}) missing from current font.
  fig.savefig(plot_fname, dpi=250)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:427: UserWarning: Matplotlib currently does not support Bengali natively.
  fig.savefig(plot_fname, dpi=250)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:427: UserWarning: Glyph 2438 (\N{BENGALI LETTER AA}) missing from current font.
  fig.savefig(plot_fname, dpi=250)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:427: UserWarning: Glyph 2439 (\N{BENGALI LETTER I}) missing from current font.
  fig.savefig(plot_fname, dpi=250)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:427: UserWarning: Glyph 2442 (\N{BENGALI LETTER UU}) missing from current font.
  fig.savefig(plot_fname, dpi=250)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:427: UserWarning: G

Speed: 0.2ms preprocess, 15.8ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/train


In [22]:
%cd /kaggle/working/

/kaggle/working


In [23]:
offset = random.random() / 5
print(f"lr: {lr}")

# simulating a training run
for epoch in range(2, epochs):
    acc = 1 - 2**-epoch - random.random() / epoch - offset
    loss = 2**-epoch + random.random() / epoch + offset
    print(f"epoch={epoch}, accuracy={acc}, loss={loss}")
    wandb.log({"accuracy": acc, "loss": loss})

run.log_code()

lr: 0.01
epoch=2, accuracy=0.20213842822483913, loss=0.6291701607204321
epoch=3, accuracy=0.6198100462640026, loss=0.46430927742787914
epoch=4, accuracy=0.6673820953323868, loss=0.42733401756370276
epoch=5, accuracy=0.7392030844792048, loss=0.29545376113548083
epoch=6, accuracy=0.7182602897858186, loss=0.33586151783756546
epoch=7, accuracy=0.7512050074353631, loss=0.21695941950496733
epoch=8, accuracy=0.7327338541753374, loss=0.25008674488942606
epoch=9, accuracy=0.8013284667658302, loss=0.2719204126348141


In [24]:
import os

# Define the path of the file to be deleted
file_path = "/kaggle/working/yolov9/bdsl_test.zip"

# Check if the file exists
if os.path.exists(file_path):
    # Delete the file
    os.remove(file_path)
    print(f"File {file_path} has been deleted.")
else:
    print(f"The file {file_path} does not exist.")


The file /kaggle/working/yolov9/bdsl_test.zip does not exist.


In [25]:
# !zip -r file.zip /kaggle/working/yolov9/runs

In [26]:
# !ls

In [27]:
# %cd /kaggle/working/
# from IPython.display import FileLink
# FileLink('/kaggle/working/yolov9/file.zip')

In [28]:
!zip -r file.zip /kaggle/working/yolov9/runs

  adding: kaggle/working/yolov9/runs/ (stored 0%)
  adding: kaggle/working/yolov9/runs/detect/ (stored 0%)
  adding: kaggle/working/yolov9/runs/detect/train/ (stored 0%)
  adding: kaggle/working/yolov9/runs/detect/train/confusion_matrix_normalized.png (deflated 27%)
  adding: kaggle/working/yolov9/runs/detect/train/val_batch0_pred.jpg (deflated 9%)
  adding: kaggle/working/yolov9/runs/detect/train/P_curve.png (deflated 11%)
  adding: kaggle/working/yolov9/runs/detect/train/R_curve.png (deflated 10%)
  adding: kaggle/working/yolov9/runs/detect/train/weights/ (stored 0%)
  adding: kaggle/working/yolov9/runs/detect/train/weights/best.pt (deflated 8%)
  adding: kaggle/working/yolov9/runs/detect/train/weights/epoch2.pt (deflated 31%)
  adding: kaggle/working/yolov9/runs/detect/train/weights/epoch4.pt (deflated 31%)
  adding: kaggle/working/yolov9/runs/detect/train/weights/epoch1.pt (deflated 31%)
  adding: kaggle/working/yolov9/runs/detect/train/weights/last.pt (deflated 8%)
  adding: kaggl

In [29]:
# from ultralytics import YOLO

# # Load a pretrained YOLOv8n model
# model = YOLO("/kaggle/working/yolov9/runs/detect/train2/weights/best.pt")

# # Run inference on 'bus.jpg' with arguments
# model.predict("/kaggle/working/yolov9/bdsl_test/test/images", save=True, imgsz=320, conf=0.5)

In [30]:
# from IPython.display import Image

# # Define the path to the image file
# image_path = "/kaggle/working/yolov9/runs/detect/predict2/এ_243.jpg"

# # Display the image
# Image(filename=image_path)


In [31]:
# import os
# from IPython.display import display, Image
# from PIL import Image as PILImage

# # Specify the directory containing the images
# directory = '/kaggle/working/yolov9/runs/detect/predict3'

# # List all files in the directory
# files = os.listdir(directory)

# # Filter out only the image files (you can add more extensions if needed)
# image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]

# # Loop through the list of image files and display each one
# for image_file in image_files:
#     image_path = os.path.join(directory, image_file)
#     display(PILImage.open(image_path))

# # Alternatively, using IPython.display.Image to directly display from the path
# for image_file in image_files:
#     image_path = os.path.join(directory, image_file)
#     display(Image(filename=image_path))
